In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.display import Audio
import random
import whisper
import json
import librosa
import plotly.graph_objects as go
from data_loader import *
from utils import *
import openpyxl
import os
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dataset_manager_first = SoundDatasetFirstTime("../Dataset")
df_train, df_val = dataset_manager_first.load_datasets()
print(f"Train dataset: {df_train.shape} samples")
print(f"Validation dataset: {df_val.shape} samples")

Train dataset: (15000, 3) samples
Validation dataset: (2000, 3) samples


In [5]:
dataset_manager = SoundDataset("../Dataset")
train_dataset, val_dataset = dataset_manager.load_datasets()
# print info about the datasets
print(f"Train dataset: {train_dataset.shape} samples")
print(f"Validation dataset: {val_dataset.shape} samples")

# Concatenate train and validation data
combined_df = pd.concat([train_dataset, val_dataset], ignore_index=True)
df_train, temp_data = train_test_split(combined_df, test_size=0.30, stratify=combined_df['label'], random_state=42)
df_val, df_test = train_test_split(temp_data, test_size=1/3, stratify=temp_data['label'], random_state=42)

# Check the sizes
print(f"Train size: {len(df_train)}")
print(f"Validation size: {len(df_val)}")
print(f"Test size: {len(df_test)}")

df_train.head()

Train dataset: (15000, 6) samples
Validation dataset: (2000, 5) samples
Train size: 11900
Validation size: 3400
Test size: 1700


,label,wav_id,audio_path,duration,word_count,text
3230,0,3230,../Dataset/train/3230.wav,3.36,11.0,in regards to epoxy glue's lock tight has bee...
6231,2,6231,../Dataset/train/6231.wav,2.97,10.0,great product ship to me fast it was as adver...
13194,2,13194,../Dataset/train/13194.wav,2.92,11.0,this is a good value for the number of enduct...
14994,2,14994,../Dataset/train/14994.wav,2.60,10.0,does a great job of cleaning all of the breasts?
15849,2,849,../Dataset/val/849.wav,2.98,NaN,good for printing may be difficult to justify...


In [8]:
# # iterate 
# for index, row in df_test.iterrows():
#     new_row = modify_path_and_id(row)
#     df_test.loc[index] = new_row    
# df_test

C:\Users\alice\AppData\Local\Temp\ipykernel_22152\574011948.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1_9764' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_test.loc[index] = new_row


,label,wav_id,audio_path,duration,word_count,text
9764,2,1_9764,../Dataset/all/1_9764.wav,3.250,11.0,make sure your state allows lip pin lights fo...
13881,2,1_13881,../Dataset/all/1_13881.wav,3.730,13.0,"shiny brass is not just clean brass, it is al..."
11118,2,1_11118,../Dataset/all/1_11118.wav,2.170,6.0,works great and leastay in bottom.
6890,2,1_6890,../Dataset/all/1_6890.wav,3.390,5.0,they are what he said.
798,2,1_798,../Dataset/all/1_798.wav,3.413,5.0,shoot for a great price.
...,...,...,...,...,...,...
3285,2,1_3285,../Dataset/all/1_3285.wav,2.950,7.0,grimes marble granite and concrete most excel...
10332,2,1_10332,../Dataset/all/1_10332.wav,2.830,10.0,well made sign holding up well on our office ...
6380,2,1_6380,../Dataset/all/1_6380.wav,2.710,9.0,good for scrubbing cry from a stainless steel...
6952,0,1_6952,../Dataset/all/1_6952.wav,3.520,7.0,way too big for the small clip.


In [9]:
df_train = df_train[['wav_id', 'label', 'audio_path', 'duration', 'word_count', 'text']]
df_val = df_val[['wav_id', 'label', 'audio_path', 'duration', 'word_count', 'text']]
df_test = df_test[['wav_id', 'label', 'audio_path', 'duration', 'word_count', 'text']]

# Display the first few rows to verify the change
df_train.head()

,wav_id,label,audio_path,duration,word_count,text
3230,1_3230,0,../Dataset/all/1_3230.wav,3.36,11.0,in regards to epoxy glue's lock tight has bee...
6231,1_6231,2,../Dataset/all/1_6231.wav,2.97,10.0,great product ship to me fast it was as adver...
13194,1_13194,2,../Dataset/all/1_13194.wav,2.92,11.0,this is a good value for the number of enduct...
14994,1_14994,2,../Dataset/all/1_14994.wav,2.60,10.0,does a great job of cleaning all of the breasts?
15849,2_849,2,../Dataset/all/2_849.wav,2.98,NaN,good for printing may be difficult to justify...


In [16]:
# Plots.plt_freq_labels(df_train, df_val)

### Translating audio into text

In [17]:
# import torch
# print(torch.cuda.is_available())
# !nvidia-smi

In [ ]:
# train_stats = calculate_duration_statistics(df_train)
# val_stats = calculate_duration_statistics(df_val)
# test_stats = calculate_duration_statistics(df_test)

# print(f"Train dataset - Mean: {train_stats[0]}, Median: {train_stats[1]}, Min: {train_stats[2]}, Max: {train_stats[3]}")
# print(f"Validation dataset - Mean: {val_stats[0]}, Median: {val_stats[1]}, Min: {val_stats[2]}, Max: {val_stats[3]}")
# print(f"Test dataset - Mean: {test_stats[0]}, Median: {test_stats[1]}, Min: {test_stats[2]}, Max: {test_stats[3]}")

# # Plots.plot_duration(df_train, df_val, df_test)

Train dataset - Mean: 2.85, Median: 2.77, Min: 2.0, Max: 6.53
Validation dataset - Mean: 2.87, Median: 2.81, Min: 2.0, Max: 6.25
Test dataset - Mean: 2.84, Median: 2.73, Min: 2.0, Max: 6.45


If the audio has a duration of less than 2 seconds, use Librosa to slow it down and save it
--> too difficult to understand both in the image and to transcribed

In [ ]:
# ## SLOW down the audio files that are <2sec
# slow_down_df(df_train)
# slow_down_df(df_test)
# slow_down_df(df_val)

Total files slowed down: 0
Total files slowed down: 0
Total files slowed down: 0


Denoise the audio file

In [ ]:
# ### DENOISE
# ### MAYBE MORE TIMES (?????)
# denoise_df(df_test)
# Audio(filename="../Dataset/test/100.wav")
# denoise_df(df_train)
# Audio(filename="../Dataset/test/0.wav")
# denoise_df(df_val)
# Audio(filename="../Dataset/test/11.wav")

In [10]:
data_get_info(df_train, "1_3230")

Audio 1_3230 	Label for the audio: 0 
	Text:  in regards to epoxy glue's lock tight has been a disappointment.


Now we are ready to extract the text

In [ ]:
# audio_manager = Audio2Text()

# df_train = audio_manager.transcribe_audio_files(df=df_train, batch_size=10, checkpoint_file = "text_test_denoised_ckpt.json")
# df_val = audio_manager.transcribe_audio_files(df=df_val, batch_size=10, checkpoint_file = "text_val_denoised_ckpt.json")
# df_test = audio_manager.transcribe_audio_files(df=df_test, batch_size=10, checkpoint_file = "text_test_denoised_ckpt.json")

	Loaded 2870 previous transcriptions.
Processing batch:
  Processing  ../Dataset/train/2870.wav
  Processing  ../Dataset/train/2871.wav
  Processing  ../Dataset/train/2872.wav
  Processing  ../Dataset/train/2873.wav
  Processing  ../Dataset/train/2874.wav
  Processing  ../Dataset/train/2875.wav
  Processing  ../Dataset/train/2876.wav
  Processing  ../Dataset/train/2877.wav
  Processing  ../Dataset/train/2878.wav
  Processing  ../Dataset/train/2879.wav
	Checkpoint saved! It contains 2880 transcriptions so far.
Processing batch:
  Processing  ../Dataset/train/2880.wav
  Processing  ../Dataset/train/2881.wav
  Processing  ../Dataset/train/2882.wav
  Processing  ../Dataset/train/2883.wav
  Processing  ../Dataset/train/2884.wav
  Processing  ../Dataset/train/2885.wav
  Processing  ../Dataset/train/2886.wav
  Processing  ../Dataset/train/2887.wav
  Processing  ../Dataset/train/2888.wav
  Processing  ../Dataset/train/2889.wav
	Checkpoint saved! It contains 2890 transcriptions so far.
Processi

In [12]:
# ### read the text from the checkpoint file and add it to the dataframe
# name = "val"
# ckpt_file = f"../Dataset/{name}/text_{name}_denoised_ckpt.json"
# add_text(df_val, ckpt_file)

The DataFrame already contains a 'text' column. I'll delete it and add the new one.


c:\Users\alice\Desktop\UNIBO\2_Anno\2_semestre\PW-Deep\SoundSentimentClassification\src\utils.py:309: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


AttributeError: Can only use .str accessor with string values!

In [17]:
# df_train.drop(columns=['word_count'], inplace=True)
# df_val.drop(columns=['word_count'], inplace=True)
# df_test.drop(columns=['word_count'], inplace=True)

C:\Users\alice\AppData\Local\Temp\ipykernel_22152\1103136242.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
dataset_manager.save_dataset(df_train, "train")
dataset_manager.save_dataset(df_val, "val")
dataset_manager.save_dataset(df_test, "test")

Dataset saved as train.xlsx
Dataset saved as val.xlsx
Dataset saved as test.xlsx


In [14]:
df_train = process_text(df_train)
Plots.plot_count_words(df_train)

       word_count                               text   wav_id
9072            1                           oothhalt   1_9072
994             3             not propaganda science    1_994
9676            2                        nus récloth   1_9676
4716            3              nice acting retrieval   1_4716
9616            2                          work else   1_9616
...           ...                                ...      ...
3967            2                          the year.   1_3967
9142            2                        sound video   1_9142
11314           3              john promajad company  1_11314
1363            3   contractor approved progreycock.   1_1363
10256           2                    «дезварит чудю»  1_10256

[119 rows x 3 columns]
Max word count: 86
Average word count: 8.22
